In [0]:
import shutil
from pathlib import Path

def drop_bronze_bad_records_tables(spark, catalog):
    spark.sql(f'use catalog {catalog}')
    system_schemas = ['information_schema', 'sys']
    schemas = [row.databaseName for row in spark.sql(f'SHOW SCHEMAS IN {catalog}').collect()]
    for schema in schemas:
        if schema in system_schemas or not schema.startswith('bronze'):
            continue
        tables = spark.sql(f'SHOW TABLES IN {catalog}.{schema}').filter('isTemporary = false')
        for row in tables.collect():
            if row.tableName.endswith('_bad_records'):
                spark.sql(f'DROP TABLE IF EXISTS {catalog}.{schema}.{row.tableName}')
                print(f'Dropped {catalog}.{schema}.{row.tableName}')

def remove_utility_bad_records_files(catalog):
    base = Path(f'/Volumes/{catalog}')
    for path in base.rglob('_badRecords'):
        if path.is_dir():
            shutil.rmtree(path, ignore_errors=True)
            print(f'Removed {path}')
        else:
            path.unlink(missing_ok=True)
            print(f'Removed {path}')

drop_bronze_bad_records_tables(spark, 'edsm')
remove_utility_bad_records_files('edsm')
